In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [ ]:
data = pd.read_csv('../HTML-extraction/output.csv')
data.describe()

,Unnamed: 0
count,115.000000
mean,216.286957
std,107.237298
min,1.000000
25%,143.000000
50%,263.000000
75%,306.500000
max,355.000000


In [16]:
stopWords = stopwords.words('dutch')
# pre processing data
def cleanData(sentence):
    processedList = ""
    
    # convert to lowercase, ignore all special characters - keep only alpha-numericals and spaces (not removing full-stop here)
    sentence = re.sub(r'[^A-Za-z0-9\s.]',r'',str(sentence).lower())
    sentence = re.sub(r'\n',r' ',sentence)
    
    # remove stop words
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [18]:
print(cleanData(data['body'][0]))

woensdag 21 maart gemeente heerenveen raadsverkiezingen. uitkomst bepaalt komende jaren koers heerenveen. kiezer goed bereiden 5 maart 14 maart 19 maart politieke debatten georganiseerd. hierbij vertegenwoordigers alle gemeentelijke partijen aanwezig. gaan elkaar debat zaken publiek plaatselijke belangen ingebracht. uiteraard gelegenheid avonden vragen stellen.


In [22]:
data['body'] = data['body'].map(lambda x: cleanData(x))

In [28]:
tmp_corpus = data['body'].map(lambda x: x.split('.'))
tmp_corpus

0      [woensdag 21 maart gemeente heerenveen raadsve...
1      [alle stemmen geteld,  ongeveer 60, 000 mensen...
2      [afgelopen zaterdag onze raadsleden lia aris l...
3      [carla dikfaber minister weten heerenveense co...
4      [juist toekomst bijzonder onderwijs druk staat...
5      [woensdag 21 maart gemeente heerenveen raadsve...
6      [intensieve campagneperiode spannende verkiezi...
7      [maandag 19 maart vindt heerenveen politiek de...
8      [woensdag 14 maart vindt akkrum politiek debat...
9      [maandag 5 maart vindt jubbega politiek debat ...
10     [aris fervent hardloper maakte tijdens trainin...
11     [prostitutieprobleem heerenveen groter wij had...
12     [zaterdag ontving boerenblij liefst 14 oudere ...
13     [christenunie cda heerenveen geschrokken advie...
14     [christenunie heerenveen klaar herindelingsver...
15     [aris leijendekker lijsttrekker christenunie h...
16     [naam lia heij getrouwd gerard samen we 5 kind...
17     [lijsttrekker aris leije

In [29]:
# corpus [[w1,w2,w3..],[..]]
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

  0%|          | 0/115 [00:00<?, ?it/s]

100%|██████████| 115/115 [00:00<00:00, 12988.26it/s]

In [31]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 1865
Num of words - 16108


In [33]:
phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

2019-02-22 11:52:31,137 : INFO : collecting all words and their counts


2019-02-22 11:52:31,138 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types


2019-02-22 11:52:31,164 : INFO : collected 8994 word types from a corpus of 16108 words (unigram + bigrams) and 1865 sentences


2019-02-22 11:52:31,165 : INFO : using 8994 counts as vocab in Phrases<0 vocab, min_count=25, threshold=50, max_vocab_size=40000000>


2019-02-22 11:52:31,166 : INFO : source_vocab length 8994


2019-02-22 11:52:31,228 : INFO : Phraser built with 0 phrasegrams


In [35]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

In [36]:
# shuffle corpus
def shuffle_corpus(sentences):
    shuffled = list(sentences)
    random.shuffle(shuffled)
    return shuffled

In [38]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 100
window_size = 2 # sentences weren't too long, so
epochs = 100
min_count = 2
workers = 4

# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

2019-02-22 11:54:34,593 : INFO : collecting all words and their counts


2019-02-22 11:54:34,593 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2019-02-22 11:54:34,597 : INFO : collected 2929 word types from a corpus of 16108 raw words and 1865 sentences


2019-02-22 11:54:34,598 : INFO : Loading a fresh vocabulary


2019-02-22 11:54:34,603 : INFO : effective_min_count=2 retains 2916 unique words (99% of original 2929, drops 13)


...

2019-02-22 11:54:38,007 : INFO : worker thread finished; awaiting finish of 2 more threads


2019-02-22 11:54:38,020 : INFO : worker thread finished; awaiting finish of 1 more threads


2019-02-22 11:54:38,026 : INFO : worker thread finished; awaiting finish of 0 more threads


2019-02-22 11:54:38,026 : INFO : EPOCH - 100 : training on 16108 raw words (16095 effective words) took 0.0s, 600388 effective words/s


2019-02-22 11:54:38,027 : INFO : training on a 1610800 raw words (1609500 effective words) took 3.4s, 477078 effective words/s


In [39]:
model.build_vocab(sentences=shuffle_corpus(corpus),update=True)

for i in range(5):
    model.train(sentences=shuffle_corpus(corpus),epochs=50,total_examples=model.corpus_count)

NameError: name 'random' is not defined

In [41]:
# save model
model.save('w2v_model')

2019-02-22 11:57:00,601 : INFO : saving Word2Vec object under w2v_model, separately None


2019-02-22 11:57:00,602 : INFO : not storing attribute vectors_norm


2019-02-22 11:57:00,603 : INFO : not storing attribute cum_table


2019-02-22 11:57:00,636 : INFO : saved w2v_model


In [42]:
# load word2vec model
model = Word2Vec.load('w2v_model')

2019-02-22 11:57:09,709 : INFO : loading Word2Vec object from w2v_model


2019-02-22 11:57:09,791 : INFO : loading wv recursively from w2v_model.wv.* with mmap=None


2019-02-22 11:57:09,794 : INFO : setting ignored attribute vectors_norm to None


2019-02-22 11:57:09,796 : INFO : loading vocabulary recursively from w2v_model.vocabulary.* with mmap=None


2019-02-22 11:57:09,799 : INFO : loading trainables recursively from w2v_model.trainables.* with mmap=None


2019-02-22 11:57:09,802 : INFO : setting ignored attribute cum_table to None


2019-02-22 11:57:09,804 : INFO : loaded w2v_model


In [54]:
model.most_similar(positive=['christenunie', 'lijsttrekker'])


/home/tjibbe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('49', 0.754084050655365),
 ('hardlopende', 0.7387628555297852),
 ('ondersteunde', 0.7354644536972046),
 ('brug', 0.7350203990936279),
 ('wens', 0.7188327312469482),
 ('pleit', 0.7175378799438477),
 ('raadsverkiezingen', 0.7173354625701904),
 ('namens', 0.7112743854522705),
 ('jaartje', 0.7042461633682251),
 ('verzet', 0.695603609085083)]

In [ ]:
vector = model.wv['computer']  # numpy vector of a word
vector

/home/tjibbe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('49', 0.754084050655365),
 ('hardlopende', 0.7387628555297852),
 ('ondersteunde', 0.7354644536972046),
 ('brug', 0.7350203990936279),
 ('wens', 0.7188327312469482),
 ('pleit', 0.7175378799438477),
 ('raadsverkiezingen', 0.7173354625701904),
 ('namens', 0.7112743854522705),
 ('jaartje', 0.7042461633682251),
 ('verzet', 0.695603609085083)]

In [ ]:
if t